In [1]:
# pip install bitsandbytes accelerate flash-attn

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [3]:
import torch
assert torch.cuda.is_available()

In [4]:
import sys
sys.settrace(None)
import pdb
sys.path.append('..')

In [ ]:
import bz2
import pickle
import importlib
from tqdm import tqdm


import time
from IPython.display import JSON

import copy

import psycopg
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TextStreamer, StoppingCriteria, StoppingCriteriaList, DynamicCache, OffloadedCache #, CodeGenTokenizer
from transformers.generation.logits_process import LogitsProcessorList

from datamodules.base_dataset_config import PROMPT_TEMPLATE

print(transformers.__version__)

/opt/conda/envs/development/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


4.55.0


In [6]:
### Remind to remove '.py'
# index_module = importlib.import_module('indexes.phi4_index')
#index_module = importlib.import_module('indexes.http_index')
#index_module = importlib.import_module('indexes.http_index_qwen')
index_module = importlib.import_module('indexes.qwen25_index')
#index_module = importlib.import_module('indexes.gemma3_index')
#index_module = importlib.import_module('indexes.llama_index')
index_config = getattr(index_module, 'index_config')

# model_module = importlib.import_module('models.phi4_model')
#model_module = importlib.import_module('models.qwen3_06B_model')
#model_module = importlib.import_module('models.qwen3_4B_model')
model_module = importlib.import_module('models.qwen25_3B_model')
#model_module = importlib.import_module('models.gemma3_4B_model')
#model_module = importlib.import_module('models.llama_1B_model')
model_config = getattr(model_module, 'model_config')

Loading Qwen/Qwen2.5-3B-Instruct


Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.31s/it]


In [7]:
streamer = TextStreamer(model_config.tokenizer)

In [8]:
from refactx import ConstrainedLogitsProcessor, ConstrainedStateList, \
                    ConstrainedState, DictIndex, patch_model
import refactx

In [9]:
question = '''Which city is the capital of the country where the Eiffel Tower is?'''
question = '''Who was the quarterback of the team that won Super Bowl 50?'''
question = '''Who is the lead character in the Uncharted series?'''
question = '''Who played Gandalf in the Lord of the Rings movies?'''
question = '''What was Nirvana's last studio album called?''' # TODO test with beamsearch 3. with beam search if I have only 1 allowed token the other beams could generate something illegal and than crash
# TODO understand the crash is due to beam search and abandon those illegal paths. or try to explore always different triples with beam search???


question = '''How many members were there in the original NSYNC music group?'''

question = 'Who influenced Nirvana band?'

question = 'Is Johnny Depp or Brad Pitt older?'
#question = 'Who is the mother of the director of film Polish-Russian War (Film)?' # Małgorzata Braunek
#question = 'Which film came out first, Blind Shaft or The Mask Of Fu Manchu?' # The Mask Of Fu Manchu
question = 'When did John V, Prince Of Anhalt-Zerbst\'s father die?' # 12 June 1516
#question = 'Is Johnny Depp older than Brad Pitt?'

question = '''During which years did Napoleon hold power in France?'''
question = '''In which books of A Song of Ice and Fire is Jon Snow not the Lord Commander of the Night's Watch?'''
question = '''When was Barack Obama president?'''
question = 'Which countries share a border with Spain? Use the fact command asap.'

question = 'Which Chronicles of Narnia book does not begin with "the" or "The"?'

prompted_texts = [model_config.apply_prompt_template(PROMPT_TEMPLATE, question), model_config.apply_prompt_template(PROMPT_TEMPLATE, '''During which years did Napoleon hold power in France?'''),]
#prompted_texts *= 2
prompted_texts = [prompted_texts[0]]

In [10]:
#print(len(prompted_texts))
#print(prompted_texts[0])

In [11]:
inputs = model_config.tokenizer(prompted_texts, return_tensors='pt', padding=True, padding_side='right')
inputs = inputs.to(model_config.model.device)
print(inputs['input_ids'].shape)

torch.Size([1, 765])


In [12]:
model_config.model.device

device(type='cuda', index=0)

In [13]:
torch.cuda.empty_cache()

In [14]:
patch_model(model_config.model)

In [15]:
DEBUG = False

In [16]:
num_beams = 1
num_batches = inputs['input_ids'].shape[0]

auto_streamer = streamer if num_beams * num_batches == 1 else None

states_lol = [[ConstrainedState(
                begin_pattern = model_config.switch_pattern,
                end_pattern = model_config.newline_token,
                cache_index = DictIndex(end_of_triple=index_config.index.end_of_triple),
                subtree_cache = DictIndex(end_of_triple=index_config.index.end_of_triple),
                debug=DEBUG
            ) for _ in range(num_beams)] 
                for _ in range(num_batches)]

refactx.CONSTRAINED_STATES = ConstrainedStateList(states_lol,
            num_beams=num_beams,
            num_batches = num_batches,
            pad_token_id = model_config.tokenizer.eos_token_id,
            debug = DEBUG, debug_tokenizer = model_config.tokenizer
     )

constrained_processor = ConstrainedLogitsProcessor(
    index=index_config.index,
    end_token=model_config.newline_token, states=refactx.CONSTRAINED_STATES, tokenizer=model_config.tokenizer)
logits_processor_list = LogitsProcessorList([
    constrained_processor
])

model_config.model.eval()
start = time.time()

with torch.no_grad():

    #pdb.set_trace() # b /opt/conda/envs/development/lib/python3.13/site-packages/transformers/generation/utils.py:3984
    out = model_config.model.generate(
        **inputs,
        logits_processor=logits_processor_list,
        max_new_tokens=800,
        streamer = auto_streamer,
        do_sample = False,
        temperature = None,
        top_k=None,
        num_beams=num_beams,
        #num_beam_groups = num_beams // num_batches,
        ## TODO group beam search works differently with logitsprocessors: input ids shape is group shape = num_beams / num_beam_groups
        #diversity_penalty=0.5,
        #num_return_sequences=1,
        num_return_sequences=num_beams,
        #no_repeat_ngram_size=1,
        #remove_invalid_values=True,
        #stopping_criteria=stopping_criteria,
        use_cache=True,
        top_p=None,
        min_p=None,
        #past_key_values=past_key_values,
        #kwargs = {'constrained_state': states}, # passing state
    )

print('Elapsed', time.time() - start)

<|im_start|>system
You are a helpful question-answering assistant that bases its answers on facts from a knowledge base and always respects the prompt.

The process to answer questions:

    You receive an input question.

    You determine the reasoning path needed to answer the question based on the information available.

    You determine the kind of answer you are asked. It can be a yes/no, a single entity, or a list of entities. Pay attention to the questions whose answer is a list of entities (e.g. Which countries share a border with Spain?): you need to find all the answer entities and include them all in the final answer.

    You get relevant facts with the "Fact:" command. You can rely on these facts and use them a proof for your answer.
    While getting facts you continue the reasoning explaining it step by step.

    Often description or short description may be useful for answering questions.

    You conclude with a concise answer that depending on the question can be a

In [34]:
torch.cuda.empty_cache()

In [35]:
_from = len(inputs.input_ids[0]) # 0
for i in range(out.shape[0]):
    print('-'*30, sum(out[i][_from:]), len(out[i][_from:]))
    print(model_config.tokenizer.decode(out[i][_from:]))

------------------------------ tensor(4747750, device='cuda:1') 800
Reasoning: To answer this question, I need to identify the Chronicles of Narnia books and determine which one does not begin with "the" or "The". I will need to find the titles of the books and check their beginnings.
Let's start by finding the titles of the Chronicles of Narnia books.
Fact: <Chronicles of Narnia (video game series)> <description> <video game series> .
Fact: <The Chronicles of Narnia (TV series)> <description> <British television series> .
Fact: <The Chronicles of Narnia (film series)> <description> <21st-century film series adaptation of the 1950s children's fantasy novels> .
Fact: <The Lion, the Witch and the Wardrobe> <description> <children's fantasy novel by C. S. Lewis, set in Narnia> .
Fact: <Prince Caspian> <description> <fantasy novel by C. S. Lewis, set in Narnia> .
Fact: <The Voyage of the Dawn Treader> <description> <children's fantasy novel by C. S. Lewis, set in Narnia> .
Fact: <The Silve

In [ ]:
for i, triple in enumerate(refactx.CONSTRAINED_STATES[0][0].generated_triples):
    print(i, model_config.tokenizer.decode(triple)[:-1], triple, end='\n')

0  <Chronicles of Narnia (video game series)> <description> <video game series> . [366, 67753, 4342, 315, 451, 1885, 685, 320, 9986, 1809, 4013, 16018, 366, 4684, 29, 366, 9986, 1809, 4013, 29, 659, 198]
1  <The Chronicles of Narnia (TV series)> <description> <British television series> . [366, 785, 65316, 315, 451, 1885, 685, 320, 15653, 4013, 16018, 366, 4684, 29, 366, 51861, 12425, 4013, 29, 659, 198]
2  <The Chronicles of Narnia (film series)> <description> <21st-century film series adaptation of the 1950s children's fantasy novels> . [366, 785, 65316, 315, 451, 1885, 685, 320, 30155, 4013, 16018, 366, 4684, 29, 366, 17, 16, 267, 33357, 4531, 4013, 33085, 315, 279, 220, 16, 24, 20, 15, 82, 2841, 594, 18378, 31863, 29, 659, 198]
3  <The Lion, the Witch and the Wardrobe> <description> <children's fantasy novel by C. S. Lewis, set in Narnia> . [366, 785, 32099, 11, 279, 38350, 323, 279, 26640, 32836, 29, 366, 4684, 29, 366, 5864, 594, 18378, 11514, 553, 356, 13, 328, 13, 20607, 11, 73

In [ ]:
for batch_i in range(num_batches):
    for beam_i in range(num_beams):
        for triple in refactx.CONSTRAINED_STATES[batch_i, beam_i].generated_triples:
            print(batch_i, beam_i, model_config.tokenizer.decode(triple)[:-1], triple, end='\n')

0 0  <The Chronicles of Narnia> <description> <series of children's fantasy novels by C. S. Lewis, 1950–1956> . [366, 785, 65316, 315, 451, 1885, 685, 29, 366, 4684, 29, 366, 19880, 315, 2841, 594, 18378, 31863, 553, 356, 13, 328, 13, 20607, 11, 220, 16, 24, 20, 15, 4142, 16, 24, 20, 21, 29, 659, 198]
0 0  <The Lion, the Witch and the Wardrobe> <publication date> <1950-01-01T00:00:00Z> . [366, 785, 32099, 11, 279, 38350, 323, 279, 26640, 32836, 29, 366, 71160, 2400, 29, 366, 16, 24, 20, 15, 12, 15, 16, 12, 15, 16, 51, 15, 15, 25, 15, 15, 25, 15, 15, 57, 29, 659, 198]
0 0  <Prince Caspian> <publication date> <1951-10-15T00:00:00Z> . [366, 67932, 11054, 79, 1103, 29, 366, 71160, 2400, 29, 366, 16, 24, 20, 16, 12, 16, 15, 12, 16, 20, 51, 15, 15, 25, 15, 15, 25, 15, 15, 57, 29, 659, 198]
0 0  <The Voyage of the Dawn Treader> <publication date> <1952-09-15T00:00:00Z> . [366, 785, 95272, 315, 279, 34507, 350, 11149, 29, 366, 71160, 2400, 29, 366, 16, 24, 20, 17, 12, 15, 24, 12, 16, 20, 51, 1

In [23]:
_from = 0
for i in range(out.shape[0]):
    print('-'*30, sum(out[i][_from:]), len(out[i][_from:]))
    print(model_config.tokenizer.decode(out[i][_from:]))

------------------------------ tensor(8430311, device='cuda:1') 1168
<|im_start|>system
You are a helpful question-answering assistant that bases its answers on facts from a knowledge base and always respects the prompt.

The process to answer questions:

    You receive an input question.

    You determine the reasoning path needed to answer the question based on the information available.

    You detemine the kind of answer you are asked. It can be a yes/no, a single entity, or a list of entities. Pay attention to the questions whose answer is a list of entities (e.g. Which countries share a border with Spain?): you need to find all the answer entities and include them all in the final answer.

    You get relevant facts with the "Fact:" command. You can rely on these facts and use them a proof for your answer.
    While getting facts you continue the reasoning explaining it step by step.

    Often description or short description may be useful for answering questions.

    You co

# Debug unconstrained

In [66]:
num_beams = 1

auto_streamer = streamer if num_beams == 1 else None

model_config.model.eval()
start = time.time()

with torch.no_grad():

    #pdb.set_trace()
    out = model_config.model.generate(
        **inputs,
        #logits_processor=logits_processor_list,
        max_new_tokens=800,
        streamer = auto_streamer,
        do_sample = False,
        temperature = 0.0,
        top_k=None,
        num_beams=num_beams,
        #num_beam_groups = num_beams // 2,
        ## TODO group beam search works differently with logitsprocessors: input ids shape is group shape = num_beams / num_beam_groups
        #diversity_penalty=0.5,
        #num_return_sequences=1,
        num_return_sequences=num_beams,
        #no_repeat_ngram_size=1,
        #remove_invalid_values=True,
        #stopping_criteria=stopping_criteria,
        use_cache=True,
        #past_key_values=past_key_values,
        #kwargs = {'constrained_state': states}, # passing state
    )

<|im_start|>system
You are a helpful question-answering assistant that bases its answers on facts from a knowledge base and always respects the prompt.

# Tools

You have to call the get_fact function to answer the questions.

You are provided with the function signature within <tools></tools> XML tags:
<tools>
{"type": "function", "function": {"name": "get_fact", "description": "Gets a fact relevant for the question.", "parameters": {"type": "object", "properties": {"description": {"type": "string", "description": "The description of the fact to get"}}, "required": ["description"]}}}
</tools>

For each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:
<tool_call>
{"name": "get_fact",  {"arguments": {"description": "Description of the fact to get"}}
</tool_call>

The process to answer questions:

    You receive an input question.

    You determine the reasoning path needed to answer the question based on the information avai

# Debug tools

In [45]:
def get_fact(description: str):
    """
    Gets a fact relevant for the question.

    Args:
        description: The description of the fact to get
    """
    return "Fact:"

tools = [get_fact]    

chat = [
    {"role": "user", "content": "Hey, what's the weather like in Paris right now?"}
]

tool_prompt = model_config.tokenizer.apply_chat_template(
    chat, 
    tools=tools,
    add_generation_prompt=True,
    return_tensors="pt",
    #return_dict=True,
    tokenize=False,
)

In [46]:
print(tool_prompt)

<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.

# Tools

You may call one or more functions to assist with the user query.

You are provided with function signatures within <tools></tools> XML tags:
<tools>
{"type": "function", "function": {"name": "get_fact", "description": "Gets a fact relevant for the question.", "parameters": {"type": "object", "properties": {"description": {"type": "string", "description": "The description of the fact to get"}}, "required": ["description"]}}}
</tools>

For each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:
<tool_call>
{"name": <function-name>, "arguments": <args-json-object>}
</tool_call><|im_end|>
<|im_start|>user
Hey, what's the weather like in Paris right now?<|im_end|>
<|im_start|>assistant



In [35]:
tool_prompt.keys()

dict_keys(['input_ids', 'attention_mask'])

In [ ]:
'''
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.

# Tools

You may call one or more functions to assist with the user query.

You are provided with function signatures within <tools></tools> XML tags:
<tools>
{"type": "function", "function": {"name": "get_fact", "description": "Gets a fact relevant for the question.", "parameters": {"type": "object", "properties": {}}}}
</tools>

For each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:
<tool_call>
{"name": "get_fact"}
</tool_call><|im_end|>
'''

'''
<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.

# Tools

You may call one or more functions to assist with the user query.

You are provided with function signatures within <tools></tools> XML tags:
<tools>
{"type": "function", "function": {"name": "get_fact", "description": "Gets a fact relevant for the question.", "parameters": {"type": "object", "properties": {"description": {"type": "string", "description": "The description of the fact to get"}}, "required": ["description"]}}}
</tools>

For each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:
<tool_call>
{"name": <function-name>, "arguments": <args-json-object>}
</tool_call><|im_end|>
<|im_start|>user
Hey, what's the weather like in Paris right now?<|im_end|>
<|im_start|>assistant
'''

In [41]:
PROMPT_TEMPLATE

[{'role': 'system',
  'content': 'You are a helpful question-answering assistant that bases its answers on facts from a knowledge base and always respects the prompt.\n\nProcess:\n\n    You receive an input question.\n\n    You get some context about the entities in the questions using description and short description.\n\n    You determine the reasoning path needed to answer the question based on the information available.\n\n    You explicitly list relevant facts, one per line, starting with "Fact:".\n\n    You explain your reasoning step by step and provide a detailed answer, justifying it based on the facts.\n\n    You conclude with a short and concise answer that must be grounded by the facts you found.\n\nYou have to answer only basing on the facts you find in the knowledge base.\nIf you cannot find relevant facts in the knowledge base to support an answer you stop and you reply: "I don\'t know.".\nAfter generating facts, please reason on top of the facts you find and avoid gener

In [55]:
TOOL_PROMPT_TEMPLATE = [
    {
        'role': 'system',
        'content': '''You are a helpful question-answering assistant that bases its answers on facts from a knowledge base and always respects the prompt.

# Tools

You have to call the get_fact function to answer the questions.

You are provided with the function signature within <tools></tools> XML tags:
<tools>
{"type": "function", "function": {"name": "get_fact", "description": "Gets a fact relevant for the question.", "parameters": {"type": "object", "properties": {"description": {"type": "string", "description": "The description of the fact to get"}}, "required": ["description"]}}}
</tools>

For each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:
<tool_call>
{"name": "get_fact",  {"arguments": {"description": "Description of the fact to get"}}
</tool_call>

The process to answer questions:

    You receive an input question.

    You determine the reasoning path needed to answer the question based on the information available.

    You detemine the kind of answer you are asked. It can be a yes/no, a single entity, or a list of entities. Pay attention to the questions whose answer is a list of entities (e.g. Which countries share a border with Spain?): you need to find all the answer entities and include them all in the final answer.

    You get relevant facts with the "get_fact" function.

    You can get some general context about the entities in the questions with "get_fact" asking for a description or a short description.

    While getting facts you continue the reasoning explaining it step by step.

    You conclude with a concise answer that depending on the question can be a yes/no, a single entity, or a list of entities. Pay attention to the questions whose answer is a list of entities.

You have to answer only basing on the facts you find in the knowledge base.
If you cannot find relevant facts in the knowledge base to support an answer you stop and you reply: "I don't know.".
After generating facts, please reason on top of the facts you find and avoid generating lot of useless facts.

You must always follow these instructions precisely and ensure your responses adhere strictly to this prompt.'''
    },
            {
                'role': 'user',
                'content': '''Which mountain is taller between Mont Blanc and Mount Rainier?
'''
            },
            {
                'role': 'assistant',
                'content': '''Reasoning: I need to provide the height of Mont Blanc and the height of Mount Rainier, then I need to compare the two heights. The final answer will be the taller mountain.
<tool_call>
{"name": "get_fact",  {"arguments": {"description": "Height of Mont Blanc"}}
</tool_call>
Fact: <Mont Blanc> <elevation above sea level> <4,807.02±0.5 meters> .
Now I need the height of Mount Rainier.
<tool_call>
{"name": "get_fact",  {"arguments": {"description": "Height of Mount Rainier"}}
</tool_call>
Fact: <Mount Rainier> <elevation above sea level> <4,389 meters> .
I found that Mont Blanc is 4,807 meters tall, while Mount Rainier is 4,389 meters, so Mont Blanc is taller than Mount Rainier.

Answer: Mont Blanc.
'''
            },
            {
                'role': 'user',
                'content': '''When was the director of Slumdog Millionaire born?
'''
            },
            {
                'role': 'assistant',
                'content': '''Reasoning: To answer this question, I need to find who is the director of Slumdog Millionaire and then his birth date. The final answer will be his birth date.
<tool_call>
{"name": "get_fact",  {"arguments": {"description": "Director of Slumdog Millionaire"}}
</tool_call>
Fact: <Slumdog Millionaire> <director> <Danny Boyle>.
Now I need the birth date of Danny Boyle.
<tool_call>
{"name": "get_fact",  {"arguments": {"description": "Birth date of Danny Boyle"}}
</tool_call>
Fact: <Danny Boyle> <date of birth> <1956-10-20T00:00:00Z>.
The director of Slumdog Millionaire is Danny Boyle, who was born on October 20, 1956, so this is the answer to the question.

Answer: October 20, 1956.
'''
            },
            {
                'role': 'assistant',
                'content': '''Reasoning: To determine if Johnny Depp is older than Brad Pitt, I need to find their respective birth years. Then I need to compare them and determine who is older. The final answer will be yes if Johnny Depp is older, no elsewise.
<tool_call>
{"name": "get_fact", "arguments": {"description": "Birth date of Johnny Depp"}}
</tool_call>
Fact: <Johnny Depp> <date of birth> <1963-06-09T00:00:00Z> .
Now I need the birth date of Brad Pitt.
<tool_call>
{"name": "get_fact", "arguments": {"description": "Birth date of Brad Pitt"}}
</tool_call>
Fact: <Brad Pitt> <date of birth> <1963-12-18T00:00:00Z> .
Johnny Depp was born on June 9, 1963, while Brad Pitt was born on December 18, 1963. So Johnny Depp is older than Brad Pitt and the answer is yes.

Answer: Yes.
'''
    },
    {
        "role": "user",
        "content": "Which countries share a border with Spain?"
    },
    {
        "role": "assistant",
        "content": '''Reasoning: To answer this question, I need to find all the countries that have a land border with Spain. They may be several. The answer will be the list of all these countries.
<tool_call>
{"name": "get_fact", "arguments": {"description": "Country bordering Spain"}}
</tool_call>
Fact: <Spain> <shares border with> <Portugal> .
<tool_call>
{"name": "get_fact", "arguments": {"description": "Country bordering Spain"}}
</tool_call>
Fact: <Spain> <shares border with> <France> .
<tool_call>
{"name": "get_fact", "arguments": {"description": "Country bordering Spain"}}
</tool_call>
Fact: <Spain> <shares border with> <Andorra> .
I found Portugal, France, and Andorra. But let me see if it there are more countries that share a border with Spain.
<tool_call>
{"name": "get_fact", "arguments": {"description": "Country bordering Spain"}}
</tool_call>
Fact: <Spain> <shares border with> <Gibraltar> .
<tool_call>
{"name": "get_fact", "arguments": {"description": "Country bordering Spain"}}
</tool_call>
Fact: <Spain> <shares border with> <Morocco> .
<tool_call>
{"name": "get_fact", "arguments": {"description": "Country bordering Spain"}}
</tool_call>
Fact: <Spain> <diplomatic relation> <Italy> .
This fact is about diplomatic relations and not borders.
<tool_call>
{"name": "get_fact", "arguments": {"description": "Country bordering Spain"}}
</tool_call>
Fact: <Spain> <diplomatic relation> <Greece> .
Also this fact is not about borders. I'm not able to find other countries bordering with Spain.
Spain shares borders with Portugal, France, Andorra, Gibraltar, and Morocco.

Answer: Portugal, France, Andorra, Gibraltar, Morocco.'''
    },
    {
        'role': 'user',
        'content': '''Has Hillary Clinton ever served as President of the United States?'''
    },
    {
        'role': 'assistant',
        'content': '''Reasoning: To answer this question, I need to find out if Hillary Clinton has ever held the position of President of the United States. If yes, the answer will be yes.
<tool_call>
{"name": "get_fact", "arguments": {"description": "Hillary Clinton president of the United States"}}
</tool_call>
Fact: <Hillary Clinton> <position held> <United States Secretary of State> .
This fact tells me that Hillary Clinton was the United States Secretary of State, but it doesn't answer the question about her presidency.
<tool_call>
{"name": "get_fact", "arguments": {"description": "Hillary Clinton president of the United States"}}
</tool_call>
Fact: <Hillary Clinton> <occupation> <Politician> .
This facts tells she is a Politician but it is not enough.
<tool_call>
{"name": "get_fact", "arguments": {"description": "Hillary Clinton president of the United States"}}
</tool_call>
Fact: <Hillary Clinton> <occupation> <Diplomat> .
This facts tells she is a Diplomat but not that she served as President.
I don't have any facts that directly answer the question about Hillary Clinton's presidency, so I assume Hillary Clinton never served as President of the United States. The answer is no.

Answer: No.'''
    },
    {
        'role': 'user',
        'content': 'During which years did Napoleon hold power in France?'
    },
    {
        'role': 'assistant',
        'content': '''Reasoning: To determine the years Napoleon held power in France, I need to identify the periods during which he held power in France. Maybe I can get this information from Napoleon description. The answer will be the time frame in which Napoleon help power in France. 
<tool_call>
{"name": "get_fact", "arguments": {"description": "Napoleon description"}}
</tool_call>
Fact: <Napoleon Bonaparte> <description> <French military leader, French Emperor 1804–1814 and again in 1815> .
Based on the description, Napoleon Bonaparte held power in France as Emperor from 1804 to 1814, and in 1815.

Answer: From 1799 to 1814, and in 1815.'''
    },
    {
        'role': 'user',
        'content': '''In which books of A Song of Ice and Fire is Jon Snow not the Lord Commander of the Night's Watch?'''
    },
    {
        'role': 'assistant',
        'content': '''Reasoning: To answer this question, I need to consider all the books in the A Song of Ice and Fire series by George R.R. Martin and identify the ones where Jon Snow is not the Lord Commander of the Night's Watch.
<tool_call>
{"name": "get_fact", "arguments": {"description": "Book in which Jon Snow is not the Lord Commander of the Night's Watch"}}
</tool_call>
Fact: <A Game of Thrones> <description> <1996 novel by George R. R. Martin> .
This fact doesn't give me any information about Jon Snow.
<tool_call>
{"name": "get_fact", "arguments": {"description": "Book in which Jon Snow is not the Lord Commander of the Night's Watch"}}
</tool_call>
Fact: <A Game of Thrones> <publication date> <1996-08-06T00:00:00Z> .
This fact is not giving any information about Jon Snow. I think the publication date won't help me.
<tool_call>
{"name": "get_fact", "arguments": {"description": "Book in which Jon Snow is not the Lord Commander of the Night's Watch"}}
</tool_call>
Fact: <A Clash of Kings> <publication date> <1998-11-16T00:00:00Z> .
These facts don't contain any information about Jon Snow. I'll try one more time.
<tool_call>
{"name": "get_fact", "arguments": {"description": "Book in which Jon Snow is not the Lord Commander of the Night's Watch"}}
</tool_call>
Fact: <A Storm of Swords> <publication date> <2000-08-08T00:00:00Z> .
I'm not able to find the facts about the books in which Jon Snow is not the Lord Commander, so I cannot give a grounded answer.

Answer: I don't know.'''
    },
]